[Memory](https://langchain-ai.github.io/langgraph/concepts/memory/)

#### What is Memory?

有两种 Short-term memory 和 Long-term memory。

Short-term memory： 就是 thread 级别的 memory，通过 checkpointer 存储在 state 中。

Long-term memory：跨 thread 的 memory，通过 stores 存储。

#### Short-term memory

##### Managing long conversation history

管理短期记忆是一种在精确性和召回率与应用程序的其他性能要求（延迟和成本）之间进行平衡的实践。

提到了两种技术：

**Editing message lists**: How to think about trimming and filtering a list of messages before passing to language model.

**Summarizing past conversations**: A common technique to use when you don't just want to filter the list of messages.

##### Editing message lists

The most direct approach is to remove old messages from a list (similar to a least-recently used cache).

![Filter Mesages](https://langchain-ai.github.io/langgraph/concepts/img/memory/filter.png)

In [ ]:
# 看上去 reducer 的第一个参数是 state 对应 key 的values，第二个参数是 node 返回的 updates
def manage_list(existing: list, updates: Union[list, dict]):
    if isinstance(updates, list):
        # Normal case, add to the history
        return existing + updates
    elif isinstance(updates, dict) and updates["type"] == "keep":
        # You get to decide what this looks like.
        # For example, you could simplify and just accept a string "DELETE"
        # and clear the entire list.
        return existing[updates["from"]:updates["to"]]
    # etc. We define how to interpret updates

class State(TypedDict):
    my_list: Annotated[list, manage_list]

def my_node(state: State):
    return {
        # We return an update for the field "my_list" saying to
        # keep only values from index -5 to the end (deleting the rest)
        "my_list": {"type": "keep", "from": -5, "to": None}
    }

In [ ]:
from langchain_core.messages import RemoveMessage, AIMessage
from langgraph.graph import add_messages
# ... other imports

class State(TypedDict):
    # add_messages will default to upserting messages by ID to the existing list
    # if a RemoveMessage is returned, it will delete the message in the list by ID
    messages: Annotated[list, add_messages]

def my_node_1(state: State):
    # Add an AI message to the `messages` list in the state
    return {"messages": [AIMessage(content="Hi")]}

def my_node_2(state: State):
    # Delete all but the last 2 messages from the `messages` list in the state
    delete_messages = [RemoveMessage(id=m.id) for m in state['messages'][:-2]]
    return {"messages": delete_messages}

##### Summarizing past conversations

![summarize](https://langchain-ai.github.io/langgraph/concepts/img/memory/summary.png)

In [ ]:
from langgraph.graph import MessagesState
class State(MessagesState):
    summary: str

In [ ]:
def summarize_conversation(state: State):

    # First, we get any existing summary
    summary = state.get("summary", "")

    # Create our summarization prompt
    if summary:

        # A summary already exists
        summary_message = (
            f"This is a summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )

    else:
        summary_message = "Create a summary of the conversation above:"

    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)

    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

##### Knowing when to remove messages

In [ ]:
from langchain_core.messages import trim_messages
trim_messages(
    messages,
    # Keep the last <= n_count tokens of the messages.
    strategy="last",
    # Remember to adjust based on your model
    # or else pass a custom token_encoder
    token_counter=ChatOpenAI(model="gpt-4"),
    # Remember to adjust based on the desired conversation
    # length
    max_tokens=45,
    # Most chat models expect that chat history starts with either:
    # (1) a HumanMessage or
    # (2) a SystemMessage followed by a HumanMessage
    start_on="human",
    # Most chat models expect that chat history ends with either:
    # (1) a HumanMessage or
    # (2) a ToolMessage
    end_on=("human", "tool"),
    # Usually, we want to keep the SystemMessage
    # if it's present in the original history.
    # The SystemMessage has special instructions for the model.
    include_system=True,
)

#### Long-term memory

##### Storing memories

In [1]:
from langgraph.store.memory import InMemoryStore


def embed(texts: list[str]) -> list[list[float]]:
    # Replace with an actual embedding function or LangChain embeddings object
    return [[1.0, 2.0] * len(texts)]


# InMemoryStore saves data to an in-memory dictionary. Use a DB-backed store in production use.
store = InMemoryStore(index={"embed": embed, "dims": 2})
user_id = "my-user"
application_context = "chitchat"
namespace = (user_id, application_context)
store.put(
    namespace,
    "a-memory",
    {
        "rules": [
            "User likes short, direct language",
            "User only speaks English & python",
        ],
        "my-key": "my-value",
    },
)
# get the "memory" by ID
item = store.get(namespace, "a-memory")
# search for "memories" within this namespace, filtering on content equivalence, sorted by vector similarity
items = store.search(
    namespace, filter={"my-key": "my-value"}, query="language preferences"
)

##### Framework for thinking about long-term memory

提到了两个问题，但是没看明白想说啥：

What is the type of memory?

When do you want to update memories?

#### Memory types

在记忆上做了一个类比：

| Memory Type | What is Stored | Human Example              | Agent Example       |
| :---------- | :------------- | :------------------------- | :------------------ |
| Semantic    | Facts          | Things I learned in school | Facts about a user  |
| Episodic    | Experiences    | Things I did               | Past agent actions  |
| Procedural  | Instructions   | Instincts or motor skills  | Agent system prompt |

关键是怎么理解这三种记忆呢？

##### Semantic Memory

提到了两种方式：Profile 和 Collection。简单理解，前者是单一的文档，后者是多个文档。

A profile is generally just a JSON document with various key-value pairs you've selected to represent your domain.

Alternatively, memories can be a collection of documents that are continuously updated and extended over time. 

重要的一点：这两者都需要维护更新。

##### Episodic Memory

Episodic memory, in both humans and AI agents, involves recalling past events or actions.

In practice, episodic memories are often implemented through few-shot example prompting, where agents learn from past sequences to perform tasks correctly. 

##### Procedural Memory

#### Writing memories

提到了两种方式：In the hot path、In the background。